<a href="https://colab.research.google.com/github/jamespan1118/Xenopus_signal_processing/blob/main/File_Structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Creating the folder structures from raw data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import librosa
from librosa import display
import pandas as pd
import numpy as np
import os
import shutil
from pydub import AudioSegment 
from pydub.utils import make_chunks 
from scipy.io.wavfile import write
import warnings

In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [19]:
#worksheet = gc.open('/content/drive/MyDrive/Xenopus_male_female_data/overall/Male Female Assay Log.gsheet').sheet1

#get_all_values gives a list of rows.
#rows = worksheet.get_all_values()

import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Xenopus_male_female_data/overall/Male Female Assay Log.xlsx')
df

,Date,Separation Status,Recording_Tank,Hydraphone,Folder Name,Tank_Number_male,RFID_male,Species_male,Mass_male,hCG_injected_male,Tank_Number_Female,RFID_Female,Species_Female,Mass_Female,hCG_injected_Female,Comment,Frog Camera
0,2022-06-14,Separate,1.0,1.0,20220614_F829C-F837E_F8297-F8340_F82C0-F835F,NaN,F829C,laevis,50.1,0.2505,31.0,F837E,laevis,130.3,0.6515,"only recorded for 55 minutes, terminated unexp...",NaN
1,2022-06-14,Separate,2.0,2.0,20220614_F829C-F837E_F8297-F8340_F82C0-F835F,NaN,F8297,laevis,46.0,0.2300,31.0,F8340,laevis,98.7,0.4935,"only recorded for 55 minutes, terminated unexp...",NaN
2,2022-06-14,Separate,3.0,3.0,20220614_F829C-F837E_F8297-F8340_F82C0-F835F,NaN,F82C0,laevis,56.3,0.2815,31.0,F835F,laevis,102.7,0.5135,"only recorded for 55 minutes, terminated unexp...",NaN
3,2022-06-15,Together,1.0,1.0,20220615_F829C-F837E_F8297-F8340_F82C0-F835F,NaN,F829C,laevis,50.1,0.2505,31.0,F837E,laevis,130.3,0.6515,ticking found,NaN
4,2022-06-15,Together,2.0,2.0,20220615_F829C-F837E_F8297-F8340_F82C0-F835F,NaN,F8297,laevis,46.0,0.2300,31.0,F8340,laevis,98.7,0.4935,no ticking,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,2022-08-03,Together,2.0,2.0,20220803_AD82B-AD7E3_AD804-AD827,5 (old 49),AD82B,malawi,16.9,0.0845,5 (old 49),AD7E3,malawi,32.4,0.1620,NaN,NaN
103,2022-08-03,Together,3.0,3.0,20220803_AD82B-AD7E3_AD804-AD827,5 (old 49),AD804,malawi,17.7,0.0885,5 (old 49),AD827,malawi,29.2,0.1460,NaN,NaN
104,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,2022-08-03,Separate,3.0,3.0,20220803_AD81F-AD7E5,5 (old 49),AD81F,malawi,12.2,0.0610,5 (old 49),AD7E5,malawi,20.9,0.1045,NaN,NaN


In [20]:
overall_dir = '/content/drive/MyDrive/Xenopus_male_female_data/overall'
raw_data_dir = '/content/drive/MyDrive/Xenopus_male_female_data/2022_phonotaxis_natural'

In [21]:
def parse_folder_name(folder_name):
  pair_number = folder_name.count('_')
  pair_names = []
  date = folder_name.split('_')[0]
  for i in range(0,pair_number):
    pair_name = folder_name.split('_')[i+1]
    pair_names.append(pair_name)
  #search for species
  species_list = []
  for i in pair_names:
    female_ID = i.split('-')[1]
    #species = np.array(df.loc[df['RFID_Female']==female_ID]['Species_Female'])[0]
    species = df.loc[df['RFID_Female']==female_ID]['Species_Female'].values[0]
    species_list.append(species)
  assert(len(pair_names)==len(species_list))
  return pair_number, pair_names, species_list

In [22]:
parse_folder_name('20220615_F829C-F837E_F8297-F8340_F82C0-F835F')

(3,
 ['F829C-F837E', 'F8297-F8340', 'F82C0-F835F'],
 ['laevis', 'laevis', 'laevis'])

In [23]:
def make_pair_dir(folder_name,overall_dir):
  pair_number, pair_names, species_list = parse_folder_name(folder_name)
  for i in range(0,pair_number):
    current_species = species_list[i]
    species_dir = os.path.join(overall_dir,current_species)
    pair_name = pair_names[i]
    pair_dir = os.path.join(species_dir,pair_name)
    os.makedirs(pair_dir,exist_ok=True)

In [28]:
for folder_name in os.listdir(raw_data_dir):
  try:
    #print(folder_name)
    make_pair_dir(folder_name,overall_dir)
  except Exception as e:
    print(folder_name+" Failed")
    print(e)

In [25]:
#create txt files containing links in individual's pair dir
#folder_name is the folder containing the original raw data
def create_txt_file(folder_name,df):
  folder_dir = os.path.join(raw_data_dir,folder_name)
  #loop through all audio files
  for filename in os.listdir(folder_dir):
    if (filename.endswith(".wav")and(not(filename.startswith('._')))
    and(not(filename.startswith('01-F8211')))):
      pair_ID = filename.split('-')[1]+'-'+filename.split('-')[2]
      female_ID = filename.split('-')[2]
      try:
        species = df.loc[df['RFID_Female']==female_ID]['Species_Female'].values[0]
        species_dir = os.path.join(overall_dir,species)
        pair_dir = os.path.join(species_dir,pair_ID)
        assert(os.path.exists(pair_dir))
        #put the link to the audio file in a text file in the pair_dir
        os.chdir(pair_dir)
        #first check whether rapping/ticking
        status = df.loc[df['Folder Name']==folder_name]['Separation Status'].values[0]
        if(status=='Separate'):
          file_rapping = open(pair_ID+"_rapping.txt","a")
          full_path = os.path.join(raw_data_dir,folder_name,filename)
          assert(os.path.exists(full_path))
          file_rapping.write(full_path+'\n')
          file_rapping.close()
        elif(status=='Together'):
          file_ticking = open(pair_ID+"_ticking.txt","a")
          full_path = os.path.join(raw_data_dir,folder_name,filename)
          assert(os.path.exists(full_path))
          file_ticking.write(full_path+'\n')
          file_ticking.close()
      except(IndexError):
        print("failed "+filename)
      except Exception as e:
        print('Failed to process %s. Reason: %s' % (filename, e))

In [29]:
for folder_name in os.listdir(raw_data_dir):
  create_txt_file(folder_name,df)

####Clear out functions

In [12]:
def clear_dir(folder):
  for filename in os.listdir(folder):
      file_path = os.path.join(folder, filename)
      try:
          if os.path.isfile(file_path) or os.path.islink(file_path):
              os.unlink(file_path)
          elif os.path.isdir(file_path):
              shutil.rmtree(file_path)
      except Exception as e:
          print('Failed to delete %s. Reason: %s' % (file_path, e))

def clear_four_species_dir():
  clear_dir('/content/drive/MyDrive/Xenopus_male_female_data/overall/laevis')
  clear_dir('/content/drive/MyDrive/Xenopus_male_female_data/overall/petersii')
  clear_dir('/content/drive/MyDrive/Xenopus_male_female_data/overall/poweri')
  clear_dir('/content/drive/MyDrive/Xenopus_male_female_data/overall/victorianus')
  clear_dir('/content/drive/MyDrive/Xenopus_male_female_data/overall/malawi')

In [13]:
clear_four_species_dir()

In [ ]:
clear_dir('/content/drive/MyDrive/Xenopus_male_female_data/laevis')
clear_dir('/content/drive/MyDrive/Xenopus_male_female_data/malawi')
clear_dir('/content/drive/MyDrive/Xenopus_male_female_data/petersii')
clear_dir('/content/drive/MyDrive/Xenopus_male_female_data/poweri')
clear_dir('/content/drive/MyDrive/Xenopus_male_female_data/victorianus')